In [13]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, MaxPooling2D, Flatten, Dense
from tensorflow.keras import Model
from tensorflow.keras.utils import get_file
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import numpy as np

In [2]:
def VGG16(classes):
    image_input = Input(shape=(224, 224, 3))
    
    # 224,224,3 -> 224,224,64 
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1-conv1')(image_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1-conv2')(x)
    
    # 224,224,64 -> 112,112,64
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1-pooling')(x)
    
    # 112,112,64 -> 112,112,128
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2-conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2-conv2')(x)
    
    # 112,112,128 -> 56,56,128
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2-pooling')(x)
    
    # 56,56,128 -> 56,56,256
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3-conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3-conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3-conv3')(x)
    
    # 56,56,256 -> 28,28,256
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3-pooling')(x)
    
    # 28,28,256 -> 28,28,512
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4-conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4-conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4-conv3')(x)
    
    # 28,28,512 -> 14,14,512
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4-pooling')(x)
    
    # 14,14,512 -> 14,14,512 there's no change in this block
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5-conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5-conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5-conv3')(x)
    
    # 14,14,512 -> 7,7,512
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5-pooling')(x)
    
    #Flatten, softmax, fc
    x = Flatten()(x)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(classes, activation='softmax', name='softmax')(x)
    
    model = Model(image_input,x,name = 'vgg16')
    
    return model

In [3]:
model = VGG16(1000)
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1-conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1-conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1-pooling (MaxPooling2D (None, 112, 112, 64)      0         
_________________________________________________________________
block2-conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2-conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2-pooling (MaxPooling2D (None, 56, 56, 128)       0     

In [14]:
WEIGHTS_PATH = 'vgg16_weights_tf_dim_ordering_tf_kernels.h5'

model.load_weights(WEIGHTS_PATH)
img_path = 'elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
print('Input image shape:', x.shape)

preds = model.predict(x)
print('Predicted:', decode_predictions(preds))

Input image shape: (1, 224, 224, 3)
40960/35363 [==================================] - 0s 2us/step
Predicted: [[('n02504458', 'African_elephant', 0.323967), ('n02504013', 'Indian_elephant', 0.3073574), ('n01871265', 'tusker', 0.26564506), ('n02437312', 'Arabian_camel', 0.061291706), ('n01688243', 'frilled_lizard', 0.00702472)]]
